In [1]:
import numpy as np
import tkinter as tk
from tkinter import messagebox

In [ ]:
class TicTacToe3D:
    # Contains all the game's logic allowing for the GUI to interact with these methods to update state and display the boad
    def __init__(self, board_size=4):
        self.board_size= board_sizes
        self.board=[[["" for _ in range(board_size)] for _ in range(board_size)] for _ in range(board_size)]
        self.players = ["X","O"]
        self.current_player_index=0

    # get_current_player and switch_player help make sure that players take turns alternatingly
    def get_current_player(self):
        return self.players[self.current_player_index]

    def switch_player(self):
        self.curent_player_index=1-self.current_player_index

    def is_valid_move(self, z, y, x):
        # Check if coordinates are within the board's boundaries and is not clearly occupied
        if not (0 <= z < self.board_size and 0 <= y < self.board_size and 0 <= x < self.board_size):
            return False
        
        # Check if the chosen cell is empty
        if self.board[z][y][x] != '':
            return False
        
        # If all checks pass, the move is valid
        return True
    
    def make_move(self, z,y,x):
        # Allows player to make move assuming is_valid_move function passes
        if self.is_valid_move(z,y,x):
            self.board[z][y][x]=self.get_current_player()
            return True
        else:
            return False
    
    def check_win(self,z,y,x):
        # Checks for a winning scenario
        player=self.board(z,y,x)
        if not player:
            return False
        
        # Check all possible winning lines from last move
        winning_directions=self._get_winning_directions(z,y,x)
        for direction in winning_directions:
            if self._check_line(z,y,x,direction,player):
                return True
        return False
    

    def _get_winning_directions(self,z,y,x):
        directions=[]
        # Planes
        directions.extend([(0, 1, 0), (0, 0, 1), (0, 1, 1), (0, 1, -1)]) 

        # Columns
        directions.append((1, 0, 0)) # Vertical through the layers (changes in z)

        # Add diagonals involving the Z-Axis
        if y == x: 
            directions.append((1, 1, 0)) # Diagonal YX in Z planes
        if y == (self.board_size - 1 - x): 
            directions.append((1, -1, 0)) # Anti-diagonal YX in Z planes
        if z == y: 
            directions.append((1, 1, 0)) # Diagonal ZY in X planes
        if z == (self.board_size - 1 - y): 
            directions.append((-1, 1, 0)) # Anti-diagonal ZY in X planes
        if z == x: 
            directions.append((1, 0, 1)) # Diagonal ZX in Y planes
        if z == (self.board_size - 1 - x): 
            directions.append((-1, 0, 1)) # Anti-diagonal ZX in Y planes
        if z == y and y == x: 
            directions.append((1, 1, 1)) # Space diagonal
        if z == (self.board_size - 1 - y) and y == x: 
            directions.append((-1, 1, 1)) # Space anti-diagonal
        if z == (self.board_size - 1 - y) and y == (self.board_size - 1 - x): 
            directions.append((-1, 1, -1)) # Space anti-anti-diagonal
        if z == y and y == (self.board_size - 1 - x): 
            directions.append((1, 1, -1)) # Space anti-anti-diagonal
        return list(set(directions))

    def _check_line(self, z, y, x, direction, player):
        # Check and see if a winning line has been formed
        dz, dy, dx = direction
        count = 1
        # Check forward direction
        for i in range(1, self.board_size):
            nz, ny, nx = z + i * dz, y + i * dy, x + i * dx
            if 0 <= nz < self.board_size and 0 <= ny < self.board_size and 0 <= nx < self.board_size and self.board[nz][ny][nx] == player:
                count += 1
            else:
                break
        # Check backward direction
        for i in range(1, self.board_size):
            nz, ny, nx = z - i * dz, y - i * dy, x - i * dx
            if 0 <= nz < self.board_size and 0 <= ny < self.board_size and 0 <= nx < self.board_size and self.board[nz][ny][nx] == player:
                count += 1
            else:
                break
        return count >= self.board_size


    def is_board_full(self):
    # Check and see if the board is filled (DRAW)
        for z in range(self.board_size):
            for y in range(self.board_size):
                for x in range(self.board_size):
                    if self.board[z][y][x] == '':
                        return False
        return True

    def ai_logic(self, depth, alpha, beta,maximizing, max_depth):
        # Function for alpha beta pruning




_IncompleteInputError: incomplete input (3841492503.py, line 114)

In [ ]:
class TicTacToe_GUI:
    def __init__(self,master):
        self.master=master
        master.title("3D Tic Tac Toe")
        self.game=TicTacToe3D()
        

In [ ]:
# Code to Run Tic Tac Toe Game
if __name__ == '__main__':
    root = tk.Tk()
    gui = TicTacToeGUI(root)
    root.mainloop()

NameError: name 'tk' is not defined